# Générer du texte à partir de mon dataset poème

## Importation des packages

In [2]:
import tensorflow as tf
import numpy as np 

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam


## Construire le vocabulaire des mots
je vais mettre en minuscules et diviser le texte brut en une liste de phrases :

In [3]:
# Charger le jeu de données
data = open('data/poeme.txt', "r",  encoding='utf-8').read()

# Mettre en minuscules et diviser le texte
corpus = data.lower().split("\n")

# Preview the result
print(corpus)


["je n'aime que toi... fait moi mettre genoux à terre,\xa0 ", "l’amitié\xa0 qu'elle soit heureuse ou de souffrance la fée avise l'ogre avec sa bouche énorme. ", "quand l'aurore cueille la rose de vos jours, un soupir, qui en dit long sur sa grâce. ", 'mes yeux te fixer ,', 'mon idole notre attente, ', "notre espoir je t'aime en quelques messages d'amour vous vintes ", "je vins c'est avec tout l'amour que peut contenir mon cœur, ", "emporte-moi dans tes rêves illégaux j'écris pour eux mais aussi pour toi ", 'et toi perse vielles mendiants, ', 'enfants et vieillards solitaires valentin n’est qu’un martyr ', "je l'aime à la folie : ", "a l'orée de vos yeux un visage de femme ", "disant qu'elle t'aimait dessinent nos initiales entrelacées ! ", 'plus haut que nous, que tout le plaisir sera notre quotidien. ', "j'ai besoin de toi, nul doute que je l'aime pour toujours, ", 'la mort qui m’étreint a se croiser sur ma poitrine ne se plieraient pas mes yeux brillent de maintes étincelles, ', "où,

À partir de là, je peux initialiser la classe Tokenizer et générer le dictionnaire d'index de mots :

In [4]:
# Initialiser la classe Tokenizer
tokenizer = Tokenizer()

# Générer le dictionnaire d'index de mots
tokenizer.fit_on_texts(corpus)


total_words = len(tokenizer.word_index) + 1
# Je défini le nombre total de mots. J'ajoute 1 pour l'index "0" qui n'est que le jeton de remplissage.
print(f'dictionnaire d\'index de mots: {tokenizer.word_index}')
print(f'nombre total de mots: {total_words}')



dictionnaire d'index de mots: {'de': 1, 'et': 2, 'je': 3, 'le': 4, 'la': 5, 'que': 6, 'tu': 7, 'mon': 8, 'à': 9, 'qui': 10, 'un': 11, 'les': 12, 'dans': 13, 'des': 14, 'me': 15, 'pour': 16, 'ne': 17, 'en': 18, 'mes': 19, 'plus': 20, 'ma': 21, 'au': 22, 'ton': 23, 'toi': 24, 'ce': 25, 'est': 26, 'a': 27, 'sur': 28, 'du': 29, 'une': 30, 'nous': 31, 'si': 32, 'moi': 33, 'cœur': 34, 'il': 35, 'tes': 36, 'tout': 37, 'amour': 38, 'as': 39, "c'est": 40, 'se': 41, 'te': 42, 'mais': 43, 'pas': 44, 'temps': 45, 'vous': 46, 'comme': 47, 'son': 48, "d'amour": 49, 'es': 50, 'elle': 51, 'vie': 52, 'suis': 53, 'avec': 54, 'sa': 55, "t'aime": 56, "l'amour": 57, 'sans': 58, 'ta': 59, 'on': 60, 'tous': 61, 'quand': 62, 'yeux': 63, 'par': 64, 'mots': 65, 'ses': 66, 'cette': 67, '\xa0': 68, 'nos': 69, 'ces': 70, 'notre': 71, "j'ai": 72, 'corps': 73, 'bien': 74, 'jour': 75, 'vos': 76, 'jamais': 77, 'toujours': 78, 'vers': 79, 'seul': 80, 'aux': 81, "qu'il": 82, 'belle': 83, 'où': 84, 'car': 85, 'noël': 86,

## Prétraitement de l'ensemble de données

Ensuite, Je vais générer les entrées et les étiquettes de mon modèle. Les xs ou les entrées du modèle seront des séquences rembourrées, tandis que les ys ou les étiquettes sont des tableaux one-hot encoded.

In [5]:
# Initialiser la liste des séquences
input_sequences = []

# Boucle sur chaque ligne
for line in corpus:

    # Tokeniser la ligne courante
    token_list = tokenizer.texts_to_sequences([line])[0]

    # Bouclez plusieurs fois sur la ligne pour générer les sous-phrases
    for i in range(1, len(token_list)):
    
        # Générer la sous-expression
        n_gram_sequence = token_list[:i+1]

        # Ajouter la sous-phrase à la liste des séquences
        input_sequences.append(n_gram_sequence)

# Obtenir la longueur de la ligne la plus longue
max_sequence_len = max([len(x) for x in input_sequences])

# Pad toutes les séquences
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Créez des entrées et une étiquette en divisant le dernier jeton dans les sous-phrases
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

# Convertir l'étiquette en tableaux one-hot
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)


Je peux ensuite afficher certains des exemples pour vérification de cohérence.

In [6]:
# Obtenir un exemple de phrase
sentence = corpus[0].split()
print(f'Phrase d\'exemple: {sentence}')

# Je vais initialiser la liste des jetons
token_list = []

# je recherche les indices de chaque mot et les ajouter à la liste
for word in sentence: 
    try:
        token_list.append(tokenizer.word_index[word])
    except:
        pass

# Print the token list
print(token_list)



Phrase d'exemple: ['je', "n'aime", 'que', 'toi...', 'fait', 'moi', 'mettre', 'genoux', 'à', 'terre,']
[3, 1249, 6, 116, 33, 846, 847, 9]


In [7]:
# Choisir un élément
elem_number = 5

# Afficher la liste et la phrase des jetons
print(f'List de token: {xs[elem_number]}')
print(f'decoder le token en text: {tokenizer.sequences_to_texts([xs[elem_number]])}')

# Afficher label
print(f'one-hot label: {ys[elem_number]}')
print(f'index de label: {np.argmax(ys[elem_number])}')


token list: [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0

## Construire et compiler le modèle

Ensuite, Je vais créer et compiler le modèle. J'ai placé certains des hyperparamètres en haut de la cellule de code afin que Je puisse facilement les modifier ultérieurement si Je le souhaite.

In [8]:
# Hyperparamètres
embedding_dim = 100
lstm_units = 150
learning_rate = 0.01

# Je construis le modèle
model = Sequential([
          Embedding(total_words, embedding_dim, input_length=max_sequence_len-1),
          Bidirectional(LSTM(lstm_units)),
          Dense(total_words, activation='softmax')
])

# J'utilise categorical crossentropy car il s'agit d'un problème multi-classes
model.compile(
    loss='categorical_crossentropy', 
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), 
    metrics=['accuracy']
    )

# Print the model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 653, 100)          336100    
                                                                 
 bidirectional (Bidirectiona  (None, 300)              301200    
 l)                                                              
                                                                 
 dense (Dense)               (None, 3361)              1011661   
                                                                 
Total params: 1,648,961
Trainable params: 1,648,961
Non-trainable params: 0
_________________________________________________________________


## Entrainer le modèle
Je peux maintenant entraîner le modèle. J'ai un vocabulaire relativement restreint, il ne faudra donc que quelques minutes pour compléter 500 époques.

In [9]:
# Entrainer le model
# history = model.fit(xs, ys, epochs=20)

In [10]:
total_words

3361